In [ ]:
pip install tensorflow numpy matplotlib

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os

# ==================== Configuration ====================
CONFIG = {
    'num_samples': 500,
    'image_size': (64, 64),
    'base_filters': 32,
    'depth': 3,
    'epochs': 60,
    'batch_size': 32,
    'learning_rate': 0.001,
}

# ==================== Data Generation ====================
def create_ring_mask(size=64, inner_radius=0.3, outer_radius=0.5):
    """Generate ring mask: ring=0.9 (dark), background=0.05-0.15 (light)"""
    y, x = np.ogrid[-1:1:size*1j, -1:1:size*1j]
    distance = np.sqrt(x**2 + y**2)
    
    # Ring region (inner_radius < distance <= outer_radius) = 0.9
    ring = (distance > inner_radius) & (distance <= outer_radius)
    
    # Background region = light color (0.05-0.15)
    background = np.random.uniform(0.05, 0.15, (size, size))
    
    # Combine: ring=0.9 (dark), background=light
    mask = np.where(ring, 0.9, background)
    return mask

def generate_synthetic_data(num_samples=500, image_size=(64, 64)):
    """Generate ring dataset"""
    X = np.array([create_ring_mask(image_size[0]) for _ in range(num_samples)])
    y = np.array([create_ring_mask(image_size[0]) for _ in range(num_samples)])
    
    # Data augmentation: rotation
    X = np.array([np.rot90(x, np.random.randint(0, 4)) for x in X])
    y = np.array([np.rot90(y_, np.random.randint(0, 4)) for y_ in y])
    
    # Add Gaussian noise to simulate measurement error
    X += np.random.normal(0, 0.03, X.shape)
    X = np.clip(X, 0, 1)
    
    # Normalization
    X = (X - X.min()) / (X.max() - X.min() + 1e-8)
    y = (y - y.min()) / (y.max() - y.min() + 1e-8)
    return X, y

def postprocess_prediction(pred, threshold=0.5, scale=10):
    """
    Enhance prediction results using Sigmoid function for smooth region separation
    
    Parameters:
    - pred: raw prediction (0-1 range)
    - threshold: sigmoid center threshold (default 0.5)
    - scale: sigmoid steepness (larger value = steeper, better separation)
    """
    # Sigmoid function: f(x) = 1 / (1 + exp(-scale * (x - threshold)))
    # Dark regions (>threshold) enhanced to ~1.0, light regions (<threshold) weakened to ~0.0
    result = 1.0 / (1.0 + np.exp(-scale * (pred - threshold)))
    return result

# ==================== Model Building ====================
def build_unet(input_shape=(64, 64, 1), base_filters=32, depth=3):
    """Lightweight U-Net architecture"""
    inputs = layers.Input(shape=input_shape)
    x = inputs
    skips = []
    
    # Encoder
    for i in range(depth):
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        skips.append(x)
        x = layers.MaxPooling2D((2, 2))(x)
    
    # Bottleneck
    x = layers.Conv2D(base_filters * 2**depth, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(base_filters * 2**depth, (3, 3), activation='relu', padding='same')(x)
    
    # Decoder
    for i in reversed(range(depth)):
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Concatenate()([x, skips[i]])
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
    
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
                  loss='binary_crossentropy', metrics=['mse', 'mae'])
    return model

# ==================== Main Program ====================
# Generate data
X, y = generate_synthetic_data(CONFIG['num_samples'], CONFIG['image_size'])
X = X[..., np.newaxis]
y = y[..., np.newaxis]

# Create results directory
result_dir = os.path.join('results')
os.makedirs(result_dir, exist_ok=True)

# Train model
print("Training U-Net...")
model = build_unet(input_shape=(64, 64, 1), base_filters=CONFIG['base_filters'], depth=CONFIG['depth'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X, y, epochs=CONFIG['epochs'], batch_size=CONFIG['batch_size'],
                   validation_split=0.2, callbacks=[early_stopping], verbose=1)

val_mse = history.history['val_mse'][-1]
val_mae = history.history['val_mae'][-1]
print(f"Final validation MSE: {val_mse:.4f}, MAE: {val_mae:.4f}")

# Save loss curve
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('U-Net Training Curve (Ring Detection)')
plt.savefig(os.path.join(result_dir, "training_loss.png"), dpi=100)
plt.close()

# Predict and visualize
idx = np.random.randint(0, X.shape[0])
sample_input = X[idx:idx+1]
sample_true = y[idx]
sample_pred = model.predict(sample_input)[0]
sample_pred_enhanced = postprocess_prediction(sample_pred, threshold=0.5, scale=10)

plt.figure(figsize=(16, 4))
plt.subplot(1, 4, 1)
plt.title('Input (Ring with Noise)')
plt.imshow(1-sample_input[0, :, :, 0], cmap='gray')
plt.colorbar()

plt.subplot(1, 4, 2)
plt.title('True Label')
plt.imshow(1-sample_true[:, :, 0], cmap='gray')
plt.colorbar()

plt.subplot(1, 4, 3)
plt.title('Raw Prediction')
plt.imshow(1-sample_pred[:, :, 0], cmap='gray')
plt.colorbar()

plt.subplot(1, 4, 4)
plt.title('Enhanced Prediction (Sigmoid)')
plt.imshow(1-sample_pred_enhanced[:, :, 0], cmap='gray')
plt.colorbar()

plt.tight_layout()
plt.savefig(os.path.join(result_dir, "prediction_comparison.png"), dpi=100)
plt.close()

# Save metrics
with open(os.path.join(result_dir, "metrics.txt"), "w") as f:
    f.write(f"Ring Detection U-Net with Sigmoid Enhancement\n")
    f.write(f"Val MSE: {val_mse:.4f}\n")
    f.write(f"Val MAE: {val_mae:.4f}\n")
    f.write(f"Samples: {CONFIG['num_samples']}, Epochs: {len(history.history['loss'])}\n")
    f.write(f"\nPostprocessing: Sigmoid function with threshold=0.5, scale=10\n")

print("Training completed! Results saved to: results/")

# U4EIP：基于 U-Net 的电磁特性恢复系统

一个基于卷积神经网络的电磁特性恢复系统，通过轻量级U-Net模型从模拟电磁波传播数据中精确重建介质介电常数分布。

---

## 设计原理

### 1. 问题定义
- 输入：模拟电磁波传播数据（64×64矩阵）
- 输出：地下介质介电常数分布（64×64矩阵）
- 任务：从噪声观测中检测并重建圆环特征

### 2. 系统架构

#### 数据层：输入信号生成（这里以圆环型输入信号为例）
圆环掩膜模式:
- 圆环区域：数值 = 0.9（代表高介电常数）
- 背景区域：0.05-0.15（代表低介电常数）
- 高斯噪声（σ=0.03）模拟测量误差
- 随机旋转用于数据增强

数据增强策略:
- 旋转增强：0度、90度、180度、270度旋转
- 噪声注入：高斯噪声σ=0.03增加真实感
- 归一化处理：逐样本最小-最大归一化保证稳定性

#### 模型层：轻量级U-Net（深度=3）
编码器（3层）:
  Conv2D(32) -> Conv2D(32) -> MaxPool
  Conv2D(64) -> Conv2D(64) -> MaxPool
  Conv2D(128) -> Conv2D(128) -> MaxPool

瓶颈层:
  Conv2D(256) x 2

解码器（3层，含跳接）:
  UpSample -> Concat -> Conv2D(128) x 2
  UpSample -> Concat -> Conv2D(64) x 2
  UpSample -> Concat -> Conv2D(32) x 2

输出层: Conv2D(1, activation='sigmoid')

架构优势:
- 参数数量：约0.5百万（轻量级，快速训练）
- 跳接连接保留空间特征
- Sigmoid输出在[0, 1]范围内用于二分类

#### 训练层：二值交叉熵优化
损失函数: 二值交叉熵
优化器: Adam（学习率=0.001）
正则化: 早停机制（耐心=10）
评价指标: MSE、MAE

#### 推理层：基于Sigmoid的后处理
增强函数:
$f(x) = 1 / (1 + exp(-scale × (x - threshold)))$

参数说明:
- threshold：决策边界（默认0.5）
- scale：陡峭程度（更大值 = 更陡峭的分离）
- 效果：从软决策平滑过渡到硬决策



---

### 3. 可配置架构
```
CONFIG = {
    'num_samples': 500,        # 数据集大小
    'image_size': (64, 64),    # 输入/输出分辨率
    'base_filters': 32,        # 基础滤波器数量
    'depth': 3,                # U-Net深度（3个编码器层）
    'epochs': 60,              # 最大训练轮数
    'batch_size': 32,          # 批量大小
    'learning_rate': 0.001,    # Adam学习率
}
```

### 4. 模块化代码结构
数据生成模块 -> 模型构建 -> 训练循环 -> 后处理和可视化
  (create_ring_mask)  (build_unet)  (model.fit)  (postprocess_prediction)
  (generate_synthetic)                            (visualize results)



---

## pipline

数据生成步骤:
  生成500个圆环样本(64×64)
  数据增强：随机旋转
  归一化处理：逐样本缩放

模型训练步骤:
  初始化轻量级U-Net
  二值交叉熵损失
  早停机制（耐心=10）
  恢复最优权重

性能评估步骤:
  计算验证集MSE和MAE
  绘制训练/验证损失曲线
  保存指标到metrics.txt

预测和增强步骤:
  模型推理测试样本
  基于Sigmoid的后处理
  生成对比可视化

输出生成步骤:
  training_loss.png（收敛曲线）
  prediction_comparison.png（4面板图）
  metrics.txt（性能摘要）

---

## 快速开始指南

### 安装环境
```
pip install tensorflow numpy matplotlib
```

### 运行训练
选项1：在Jupyter/IPython中运行
cd src && jupyter notebook main.ipynb

选项2：Python执行
cd src && python -c "exec(open('main.ipynb').read())"

### 查看结果
```
ls -la results/
输出：
training_loss.png          - 损失曲线收敛
prediction_comparison.png  - 4面板对比（输入、真值、原始、增强）
metrics.txt               - 性能摘要
```

---

## 预期结果

### 验证指标
- MSE：小于0.01（目标）
- MAE：小于0.05（目标）
- 收敛：通常30-50个轮次（含早停）

### 可视化组件
1. 输入（含噪声圆环）：噪声观测（0.9圆环+0.05-0.15背景）
2. 真标签：干净圆环掩膜（基准真值）
3. 原始预测：Sigmoid连续输出[0, 1]
4. 增强预测：后处理二值输出

---

## 物理解释

数值范围对应的物理含义:
0.9：高介电常数（圆环）- 材料例如：粘土、水、混凝土
0.05-0.15：低介电常数（背景）- 材料例如：空气、沙土、土壤

应用领域:
- 地面穿透雷达(GPR)：地下异常检测
- 医学成像：MRI/超声伪影移除
- 无损检测：工业缺陷检测
- 地球物理勘测：层界面识别

---

***Last Updated: 2025.12.11***  

